<a href="https://colab.research.google.com/github/jaekyoungkim/DL_of_everyone/blob/main/HAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://blog.naver.com/hist0134/221386940063
# http://hist0134.blog.me/221179965199
# https://github.com/hist0613/keras-implementations/blob/main/IMDB-HieAtt.ipynb
# https://github.com/kwonsuhan/Exchange-Rate-Caster/blob/a4dff9961e44572b7a950c347944f3386f9cbe1e/code/han/HAN_model.py
# [Keras] Hierarchical Attention Networks for Document Classification 구현

In [35]:
# IMDB-HieAtt.ipynb
# We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well. 
# 

In [39]:
!nltk.download('punkt')
import nltk

/bin/bash: -c: line 0: syntax error near unexpected token `'punkt''
/bin/bash: -c: line 0: `nltk.download('punkt')'


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
#4/1AX4XfWgKzT5FnEsEqbHv_MJeAYGD6bx-GaC_4wW5f-wWQPUEX9zI2zEL4ug
# crm = pd.read_csv('gdrive/MyDrive/DW_DATA/CRMDATA_product.csv', encoding = 'c==p949')  # cp949
# CRM품목정리 파일 활용
# utf-8  or cp 949 로 코딩

Mounted at /content/gdrive


In [129]:
MAX_SENTENCES = 10
MAX_SENTENCE_LENGTH = 25

In [23]:
# refer: http://ai.stanford.edu/~amaas/data/sentiment/


import os

data_dir = "gdrive/MyDrive/dataset/IMDB/aclImdb"  # 데이터 받아서 해당 폴더 생성한거에 집어넣음
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
print(train_dir)
print(test_dir)

gdrive/MyDrive/dataset/IMDMB/aclImdb/train
gdrive/MyDrive/dataset/IMDMB/aclImdb/test


In [27]:
split_dir = os.path.join(base_dir, 'train')
split_dir

'gdrive/MyDrive/dataset/IMDMB/aclImdb/train'

In [30]:
def load_dataset(split='train'):
    x_data = []
    y_data = []
    
    base_dir = 'gdrive/MyDrive/dataset/IMDB/aclImdb'
    split_dir = os.path.join(base_dir, split)
    for sentiment, y in [('neg', 0), ('pos', 1)]:
        data_dir = os.path.join(split_dir, sentiment)
        for file_name in os.listdir(data_dir):
            file_path = os.path.join(data_dir, file_name)
            with open(file_path, 'r', encoding='utf-8') as fp:
                review = fp.read()
            x_data.append(review)
            y_data.append(y)
            
    return x_data, y_data

In [31]:
train_x_data, train_y_data = load_dataset(split='train')
test_x_data, test_y_data = load_dataset(split='test')

print("len(train_x_data): {}".format(len(train_x_data)))
print("len(test_x_data): {}".format(len(test_x_data)))

len(train_x_data): 25000
len(test_x_data): 25000


In [49]:
train_x_data[5]

'Are you kidding me?! A show highlighting someone who opens cans and envelopes for a meal? How talented do you have to be to do this? She MAY be able to cook but it is NOT portrayed in this half-hour stomach churning painful production. I know she has a Martha-Stewart-esquire empire. So does Warren Buffett but I don\'t see him with fake knockers opening cans of cream corn and Alpo.<br /><br />She has a nephew named...Brycer. Brycer? Stop talking about anyone a name that stupid.<br /><br />More time is spent on "table-scapes" than actual cooking. Who has that kind of time?! Silicon should be on your spatula, not on my TV. This show should be on Cartoon Network, NOT Food Network.'

In [130]:
train_y_data[5]

0

In [83]:
import numpy as np
from keras.preprocessing.text import Tokenizer  # 토큰화하는 패키지
from keras.preprocessing.sequence import pad_sequences # padding 을 위한 패키지
#from keras.utils import to_categorical # 범주형 변수로 만들어줌 / 실행안되는 이유확인
from tensorflow.keras.utils import to_categorical # 
from nltk.tokenize import sent_tokenize  # 문장을 token화 시킴


In [85]:
a=sent_tokenize(train_x_data[1])

In [87]:
a  #문장단위로 끊어줌, 끊는 기준이 있을것임

['If i could have rated this movie by 0 i would have !',
 'I see some ppl at IMDb says that this is the funniest movie of the year , etc etc excuse me ?',
 'are you ppl snorting LSD or ........?',
 'There is absolutely NOTHING funny about this movie N O T H I N G !',
 'I actually want my 27 minutes back of my life that i spent watching this piece of crap.',
 '<br /><br />I read someone sitting on an airplane watching this movie stopped watching after 30 minutes , i totally understand that , i actually would have watched snakes on a plane for 2 times over instead of watching this movie once !',
 '<br /><br />DO NOT watch this movie , do something else useful with your life do the dishes , walk the dog , hell... anything is better than spending time in front of the TV watching hot rod.']

In [92]:
b = tokenizer.texts_to_sequences(a)
print(b)

[[43, 10, 98, 25, 1196, 11, 17, 31, 2196, 10, 58, 25], [10, 63, 47, 18881, 30, 869, 546, 12, 11, 6, 1, 1536, 17, 4, 1, 301, 506, 506, 1385, 68], [23, 22, 18881, 17516, 11799, 38], [46, 6, 419, 161, 152, 42, 11, 17, 3184, 1461, 789, 2150, 10, 3184, 1299], [10, 160, 178, 56, 7720, 228, 141, 4, 56, 114, 12, 10, 995, 147, 11, 412, 4, 590], [7, 7, 10, 339, 296, 1241, 20, 32, 3922, 147, 11, 17, 2339, 147, 100, 1086, 228, 10, 456, 386, 12, 10, 160, 58, 25, 287, 3202, 20, 3, 1545, 15, 230, 209, 121, 298, 4, 147, 11, 17, 281], [7, 7, 77, 21, 103, 11, 17, 77, 137, 329, 4463, 16, 125, 114, 77, 1, 13497, 1192, 1, 826, 603, 231, 6, 126, 71, 3281, 55, 8, 982, 4, 1, 240, 147, 879, 5479]]


In [95]:
c = pad_sequences(b, maxlen=25) # 길이를 25로 맞추기
print(c)

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0    43    10    98    25  1196    11    17    31  2196    10    58
     25]
 [    0     0     0     0     0    10    63    47 18881    30   869   546
     12    11     6     1  1536    17     4     1   301   506   506  1385
     68]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0    23    22 18881 17516 11799
     38]
 [    0     0     0     0     0     0     0     0     0     0    46     6
    419   161   152    42    11    17  3184  1461   789  2150    10  3184
   1299]
 [    0     0     0     0     0     0     0    10   160   178    56  7720
    228   141     4    56   114    12    10   995   147    11   412     4
    590]
 [ 1086   228    10   456   386    12    10   160    58    25   287  3202
     20     3  1545    15   230   209   121   298     4   147    11    17
    281]
 [ 4463    16   125   114    77     1 13497  1192     1   

In [69]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_x_data)
tokenizer.fit_on_texts(test_x_data)

max_nb_words = len(tokenizer.word_index) + 1  # 토큰나이즈했을때에 나타나는 단어개수만큼


In [55]:
tokenizer

In [76]:
max_nb_words # 124253

124253

In [78]:
MAX_SENTENCE_LENGTH

25

In [79]:
MAX_SENTENCES

10

In [63]:
!pip install nltk

In [66]:
import nltk
nltk.download()
# 여기서 punkt를 설치할 수 있음

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Unzipping tokenizers/punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [70]:
def doc2hierarchical(text,
                     max_sentences=MAX_SENTENCES,
                     max_sentence_length=MAX_SENTENCE_LENGTH):
    sentences = sent_tokenize(text)  
    tokenized_sentences = tokenizer.texts_to_sequences(sentences)
    tokenized_sentences = pad_sequences(tokenized_sentences, maxlen=max_sentence_length)

    pad_size = max_sentences - tokenized_sentences.shape[0]

    if pad_size <= 0:  # tokenized_sentences.shape[0] < max_sentences
        tokenized_sentences = tokenized_sentences[:max_sentences]
    else:
        tokenized_sentences = np.pad(
            tokenized_sentences, ((0, pad_size), (0, 0)),
            mode='constant', constant_values=0
        )
    
    return tokenized_sentences
            
def build_dataset(x_data, y_data, 
                  max_sentences=MAX_SENTENCES, 
                  max_sentence_length=MAX_SENTENCE_LENGTH,
                  tokenizer=tokenizer):
    
    nb_instances = len(x_data)
    X_data = np.zeros((nb_instances, max_sentences, max_sentence_length), dtype='int32')
    for i, review in enumerate(x_data):
        tokenized_sentences = doc2hierarchical(review)
            
        X_data[i] = tokenized_sentences[None, ...]
        
    nb_classes = len(set(y_data))  # 0,1 2개 카테고리 > 2값이 나옴
    Y_data = to_categorical(y_data, nb_classes)  #  1/0 binary를 2개의 컬럼으로 나타냄
    
    return X_data, Y_data

train_X_data, train_Y_data = build_dataset(train_x_data, train_y_data)
test_X_data, test_Y_data = build_dataset(test_x_data, test_y_data)

print("train_X_data.shape: {}".format(train_X_data.shape))
print("test_X_data.shape: {}".format(test_X_data.shape))

train_X_data.shape: (25000, 10, 25)
test_X_data.shape: (25000, 10, 25)


In [71]:
from sklearn.model_selection import train_test_split
train_X_data, val_X_data, train_Y_data, val_Y_data = train_test_split(train_X_data, train_Y_data, 
                                                                      test_size=0.1, 
                                                                      random_state=42)
print("train_X_data.shape: {}".format(train_X_data.shape))
print("train_Y_data.shape: {}".format(train_Y_data.shape))
print("val_X_data.shape: {}".format(val_X_data.shape))
print("val_Y_data.shape: {}".format(val_Y_data.shape))

train_X_data.shape: (22500, 10, 25)
train_Y_data.shape: (22500, 2)
val_X_data.shape: (2500, 10, 25)
val_Y_data.shape: (2500, 2)


In [74]:
train_X_data[1]

array([[    0,     0,     0,    56,   325,     2,    10,    37,     5,
          817,    62,   364,   182,   859,   861,    97,     2,   103,
           93,    16,   260,   350,    15,     3,   423],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
           73,   210,    11,    27,    20, 83582,  2295,     2,   877,
            5,   726,     9,     5,   260,  6526,  1011],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,   146,    50,    10,   131,    11,    17,     6,   372,
           10,   383,     9,     8,     3,    49,    95],
       [    0,     0,   267,    42,     9,     1,   113,   355,   158,
           64,  1367,     6,    39,    34, 12595,     2,   359,   332,
           18,  1606,    48,   162,     9,    34,    49],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,    10,   101,     8,    27,   129,     1,   150,   588,
           37,    33,    70,   160,  1334,     8,   274],


In [75]:
train_Y_data[1]

array([1., 0.], dtype=float32)

In [101]:
from sklearn.model_selection import train_test_split
# train , test 데이터 분리하기
train_X_data, val_X_data, train_Y_data, val_Y_data = train_test_split(train_X_data, train_Y_data, 
                                                                      test_size=0.1, 
                                                                      random_state=42)

print("train_X_data.shape: {}".format(train_X_data.shape))
print("train_Y_data.shape: {}".format(train_Y_data.shape))
print("val_X_data.shape: {}".format(val_X_data.shape))
print("val_Y_data.shape: {}".format(val_Y_data.shape))

train_X_data.shape: (18225, 10, 25)
train_Y_data.shape: (18225, 2)
val_X_data.shape: (2025, 10, 25)
val_Y_data.shape: (2025, 2)


In [102]:
# google이 이미 word2vec의 결과로 각 word에 대한 vector를 만들어서 배포
# 하지만 용량이 커서 작은 용량에서는 어렵죠. 그래서 작은 용량의 word2vec vector로 존재
# vector를 특정 분야에 맞게 특화시키킬 원할 경우,
# Word2vec.intersect_word2vec_format(googleNews_filepath, binary=True, lockf=1.0)을 통해 쉽게 초기값을 설정
# word2vec-GoogleNews-vectors에서 이미 학습된 word2vector를 다운가능
# 다만, 학습된 모델을 가져오는 것이 아니라, “학습된 vector”만을 수치로 가져오는 것이죠.

import gensim.downloader as api
wv = api.load('word2vec-google-news-300')
vec_king = wv['king']

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [106]:
wv['boy'].shape # 300차원 벡터로 표현됨  GoogleNews-vectors-negative300

(300,)

In [115]:
embedding_dir = 'gdrive/MyDrive/dataset/word2vec' # 파일 넣은 위치
# word2vec을 가져옴
def load_word2vec(tokenizer=tokenizer):
    from gensim.models import KeyedVectors
    embedding_path = os.path.join(embedding_dir, 'GoogleNews-vectors-negative300.bin')
    # https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz 해당링크에서 파일 다운로드 1.5GB정도됨
    embeddings_index = KeyedVectors.load_word2vec_format(embedding_path, binary=True)
    
    return embeddings_index
    

In [116]:
def load_embedding(embedding_type='word2vec',
                   tokenizer=tokenizer,
                   embedding_dim=300):
    
    if embedding_type == 'word2vec':
        embeddings_index = load_word2vec()  # 위에서 언급한 함수가 사용됨 load_word2vec
        
    embedding_matrix = np.random.normal(0, 1, (max_nb_words, embedding_dim))
    for word, i in tokenizer.word_index.items():
        try:
            embedding_vector = embeddings_index[word]
        except KeyError:
            embedding_vector = None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix


In [131]:
embedding_matrix = load_embedding('word2vec')

print("embedding_matrix.shape: {}".format(embedding_matrix.shape))

embedding_matrix.shape: (124253, 300)


In [158]:
embedding_matrix # 총 124253 개의 단어들을 각각 300차원으로 만들어서 표현함

array([[-0.36300594,  0.80858456,  1.75805115, ..., -0.58748376,
         0.3996815 ,  0.41353852],
       [ 0.08007812,  0.10498047,  0.04980469, ...,  0.00366211,
         0.04760742, -0.06884766],
       [-0.26642567, -2.07034651,  0.23595789, ..., -0.71999431,
        -1.74246022, -0.27273225],
       ...,
       [-0.33974752, -0.01249639, -0.92992407, ...,  0.11315422,
         0.25316221,  0.32326909],
       [-0.09130859, -0.28320312,  0.07128906, ..., -0.33789062,
        -0.36132812,  0.29101562],
       [-0.18066406,  0.10351562, -0.0324707 , ...,  0.19238281,
        -0.14550781,  0.0480957 ]])

In [182]:
embedding_matrix.shape[0]

124253

In [183]:
# evaluation function 3가지
def recall(y_target, y_pred):   # 2개 요소 실제값, 예측값 필요
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    
    # TP = y_target_yn * y_pred_yn
    # FN = y_target_yn - (y_target_yn * y_pred_yn) 
    
    # return a single tensor value
    return recall


def precision(y_target, y_pred):# 2개 요소 실제값, 예측값 필요
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # FP = y_pred_yn - (y_target_yn * y_pred_yn) 
    
    # return a single tensor value
    return precision


def f1score(y_target, y_pred): # 2개 요소 실제값, 예측값 필요
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [204]:
import keras
from keras import backend as K
import tensorflow as tf

#from keras.engine.topology import Layer 수정해야함
#from tensorflow.python.keras.layers import Layer, InputSpec

from tensorflow.keras.layers import Layer, InputSpec


from keras.layers import Input, Embedding, Dense
from keras.layers import Lambda, Permute, RepeatVector, Multiply
from keras.layers import Bidirectional, TimeDistributed
from keras.layers import CuDNNGRU, GRU
from keras.layers import BatchNormalization, Dropout
from keras.models import Model, Sequential

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [205]:
class AttentionLayer(Layer):  # Layer를 가져옴
    def __init__(self, attention_dim, **kwargs): #  **kwargs : list가져올때 씀
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)  # attentionlayer의 속성을 가져옴
    
    def build(self, input_shape):  
        self.W = self.add_weight(name='Attention_Weight', # attention 값 이값을 확인해야하는것 같음
                                 shape=(input_shape[-1], self.attention_dim),
                                 initializer='random_normal',
                                 trainable=True)  # 학습가능
        self.b = self.add_weight(name='Attention_Bias',
                                 shape=(self.attention_dim, ), # attention값과 쌍을 이룸?
                                 initializer='random_normal',
                                 trainable=True)
        self.u = self.add_weight(name='Attention_Context_Vector',  # u: context vector
                                 shape=(self.attention_dim, 1),  # attention값과 쌍을 이룸?
                                 initializer='random_normal',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)
        
    def call(self, x):
        # refer to the original paper
        # link: https://www.cs.cmu.edu/~hovy/papers/16HLT-hierarchical-attention-networks.pdf
        u_it = K.tanh(K.dot(x, self.W) + self.b) # hiddenlayer와의 계산
        a_it = K.dot(u_it, self.u)  # a가 attention을 의미함 word attention이라고 볼 수 있음
        a_it = K.squeeze(a_it, -1) # 일자형태로 만들기
        a_it = K.softmax(a_it)  # softmax함수 적용 합이1이 되게함
        
        return a_it
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1])
    


In [206]:

def WeightedSum(attentions, representations): # 
    # from Shape(batch_size, len_units) to Shape(batch_size, rnn_dim * 2, len_units)
    repeated_attentions = RepeatVector(K.int_shape(representations)[-1])(attentions)
    # from Shape(batch_size, rnn_dim * 2, len_units) to Shape(batch_size, len_units, lstm_dim * 2)
    repeated_attentions = Permute([2, 1])(repeated_attentions)

    # compute representation as the weighted sum of representations
    aggregated_representation = Multiply()([representations, repeated_attentions])
    aggregated_representation = Lambda(lambda x: K.sum(x, axis=1))(aggregated_representation)

    return aggregated_representation # 합쳐진 representation
    
    
def HieAtt(embedding_matrix,  # word2vec한 값
           max_sentences, 
           max_sentence_length,
           nb_classes,  # 클래스의 개수 여기서는 2개 
           embedding_dim=300,  # 임베딩 차원 여기서는 300차원으로 변환시킴
           attention_dim=100,
           rnn_dim=150,
           include_dense_batch_normalization=False,
           include_dense_dropout=True,
           nb_dense=1,
           dense_dim=300,
           dense_dropout=0.2,
           optimizer = tf.keras.optimizers.Adam(lr=0.001)):
    # Use tf.keras.optimizers.Adam(learning_rate) instead of keras.optimizers.Adam(learning_rate)

    # embedding_matrix = (max_nb_words + 1, embedding_dim)
    max_nb_words = embedding_matrix.shape[0] - 1  # 124253-1
    embedding_layer = Embedding(max_nb_words + 1, 
                                embedding_dim,   
                                weights=[embedding_matrix],
                                input_length=max_sentence_length,     # input
                                trainable=False)   # embedding한걸 다시 학습할 필요없음

    # first, build a sentence encoder
    sentence_input = Input(shape=(max_sentence_length, ), dtype='int32')
    embedded_sentence = embedding_layer(sentence_input)   # sentence에 대한 임베딩 / sentence단위의 임베딩으로 한단계 위
    embedded_sentence = Dropout(dense_dropout)(embedded_sentence)
    contextualized_sentence = Bidirectional(GRU(rnn_dim, return_sequences=True))(embedded_sentence) # CuDNNGRU
    # embedded sentence 를 bidirectional featrue2개가 생성되고 두개가 합쳐짐
    
    # word attention computation
    word_attention = AttentionLayer(attention_dim)(contextualized_sentence)  # 위에서 설정한 attentionlayer함수사용
    sentence_representation = WeightedSum(word_attention, contextualized_sentence)  # 위에서 설정한 weightedsum 함수사용
    
    sentence_encoder = Model(inputs=[sentence_input],   # from keras.models import Model,
                             outputs=[sentence_representation])  # sentence를 벡터로 표현하는 과정

    # then, build a document encoder (최종 아웃풋임)
    document_input = Input(shape=(max_sentences, max_sentence_length), dtype='int32')
    embedded_document = TimeDistributed(sentence_encoder)(document_input)
    contextualized_document = Bidirectional(GRU(rnn_dim, return_sequences=True))(embedded_document)  # CuDNNGRU
    # sentence를 이용해서 doc에 대한 representation v계산
    
    # sentence attention computation
    sentence_attention = AttentionLayer(attention_dim)(contextualized_document)  
    # 해당 코드에서 가장 중요한 부분 : sentence_attention  각 sentence별 attention값이 계산되는것을 확인
    document_representation = WeightedSum(sentence_attention, contextualized_document)
    # doc에 대해 하나로 표현가능한 vector값이 나오게됨

    # 마지막 fc layer를 통해서 분류문제를 해결하고자 함
    # finally, add fc layers for classification
    fc_layers = Sequential()
    for _ in range(nb_dense): # 클래스 수만큼 fc layer를 수행함
        if include_dense_batch_normalization == True:  #  batch norm 을 한경우
            fc_layers.add(BatchNormalization())
        fc_layers.add(Dense(dense_dim, activation='relu'))
        if include_dense_dropout == True:   # drop_out 을 한경우
            fc_layers.add(Dropout(dense_dropout))
    fc_layers.add(Dense(nb_classes, activation='softmax'))  # soft max 
    
    pred_sentiment = fc_layers(document_representation) # 최종 결과값이라 볼 수 있음

    model = Model(inputs=[document_input],
                  outputs=[pred_sentiment])
    
    ############### build attention extractor ###############
    word_attention_extractor = Model(inputs=[sentence_input],
                                     outputs=[word_attention])
    word_attentions = TimeDistributed(word_attention_extractor)(document_input)
    attention_extractor = Model(inputs=[document_input],
                                     outputs=[word_attentions, sentence_attention])
    # 모델에 대한 마지막 정의
    model.compile(loss=['categorical_crossentropy'],
              optimizer=optimizer,
              metrics=['accuracy'])

    return model, attention_extractor, sentence_encoder, sentence_attention


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [202]:
model_name = "HieAtt"
model_path = '/models/checkpoints/{}.h5'.format(model_name)
checkpointer = ModelCheckpoint(filepath=model_path,
                               monitor='val_acc',
                               verbose=True,
                               save_best_only=True,
                               mode='max')

model, attention_extractor, sent_enc, sent_att = HieAtt(embedding_matrix=embedding_matrix,
                                    max_sentences=MAX_SENTENCES,
                                    max_sentence_length=MAX_SENTENCE_LENGTH,
                                    nb_classes=2,
                                    embedding_dim=300,
                                    attention_dim=100,
                                    rnn_dim=150,
                                    include_dense_batch_normalization=False,
                                    include_dense_dropout=True,
                                    nb_dense=1,
                                    dense_dim=300,
                                    dense_dropout=0.2,
                                    optimizer = tf.keras.optimizers.Adam(lr=0.001))


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [191]:
model

In [203]:
model.summary()  # OUTPUT SHAPE를 확인 가능

Model: "model_49"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_26 (InputLayer)          [(None, 10, 25)]     0           []                               
                                                                                                  
 time_distributed_24 (TimeDistr  (None, 10, 300)     37712900    ['input_26[0][0]']               
 ibuted)                                                                                          
                                                                                                  
 bidirectional_25 (Bidirectiona  (None, 10, 300)     406800      ['time_distributed_24[0][0]']    
 l)                                                                                               
                                                                                           

In [170]:
attention_extractor

In [174]:
x=[train_X_data];type(x)

list

In [176]:
y=[train_Y_data]; type(y)

list

In [207]:
validation_data=(val_X_data, val_Y_data) ;type(validation_data) # TUPLE

tuple

In [208]:
validation_data=[val_X_data, val_Y_data] ;type(validation_data) # LIST 

list

In [218]:
# 실제 데이터에 적용하는 과정
# Hyperparameter
max_sentences = 35
max_sentence_length = 10  # maxlen이랑 같음
# 사전에서 단어 수(embedding layer에서 사용)
max_nb_words = len(tokenizer.word_index) + 1

early_stop = EarlyStopping(monitor='val_loss', patience=10)
import gc
gc.collect()

500997

In [219]:
history = model.fit(x=[train_X_data],  # x : list
                    y=[train_Y_data],  # y: list
                    batch_size=128,
                    epochs=30,
                    verbose=True,
                    validation_data=(val_X_data, val_Y_data)  # tuple , 'NoneType' object is not callable
                    )
                    #,callbacks=[early_stop])


# error
# def _fixup_shape(images, labels):
#    images.set_shape([None, 15, 256, 256, 3])
#    labels.set_shape([None, 12])
#    return images, labels
# as_list() is not defined on an unknown tensorshape



Epoch 1/30


TypeError: ignored

In [ ]:
model.load_weights(model_path)
score = model.evaluate(test_X_data, test_Y_data, verbose=0, batch_size=128)
print("Test Accuracy of {}: {}".format(model_name, score[1]))

In [ ]:
import matplotlib.pyplot as plt

history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()
acc = history_dict['acc']
val_acc = history_dict['val_acc']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
model.summary()

In [ ]:
import seaborn as sn

word_rev_index = {}
for word, i in tokenizer.word_index.items():
    word_rev_index[i] = word

def sentiment_analysis(review):        
    tokenized_sentences = doc2hierarchical(review)
    
    # word attention만 가져오기
    pred_attention = attention_extractor.predict(np.asarray([tokenized_sentences]))[0][0]
    for sent_idx, sentence in enumerate(tokenized_sentences):
        if sentence[-1] == 0:
            continue
            
        for word_idx in range(MAX_SENTENCE_LENGTH):
            if sentence[word_idx] != 0:
                words = [word_rev_index[word_id] for word_id in sentence[word_idx:]]
                pred_att = pred_attention[sent_idx][-len(words):]
                pred_att = np.expand_dims(pred_att, axis=0)
                break

        fig, ax = plt.subplots(figsize=(len(words), 1))
        plt.rc('xtick', labelsize=16)
        midpoint = (max(pred_att[:, 0]) - min(pred_att[:, 0])) / 2
        heatmap = sn.heatmap(pred_att, xticklabels=words, yticklabels=False, square=True, linewidths=0.1, cmap='coolwarm', center=midpoint, vmin=0, vmax=1)
        plt.xticks(rotation=45)
        plt.show()
        
# sentiment_analysis("Delicious healthy food. The steak is amazing. Fish and pork are awesome too. Service is above and beyond. Not a bad thing to say about this place. Worth every penny!")
sentiment_analysis("Absolute perfection end game !! Good acting performance to all the characters. Great cgi's. Truly epic & perfect ending to a long journey of marvel movie. Go 

In [ ]:
# Document classification에 사용할 데이터셋은 Yelp Reviews 입니다. sentiment classification을 하는 데이터셋인데, 각 줄마다 하나의 json 데이터가 있는 구조
x_train = []
y_train = []
with open(data_dir, 'r', encoding="utf-8") as fp:
    for line in fp:
        # ['user_id', 'stars', 'text', 'funny', 'useful', 'review_id', 'business_id', 'date', 'cool']
        line = json.loads(line)
        x_train.append(line['text'])
        y_train.append(int(line['stars']) - 1)

